<a href="https://colab.research.google.com/github/sakibreza/Emotion-Recognition-from-Multi-Channel-EEG-Signals/blob/main/DEAP_DE_Feature_Extraction_1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
import pickle as pickle
import matplotlib.pyplot as plt
import os
import sys
import math
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/MyDrive/DEAP'
os.chdir(path)
print(os.getcwd()) 

/content/drive/MyDrive/DEAP


In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
	nyq = 0.5 * fs
	low = lowcut / nyq
	high = highcut / nyq
	b, a = butter(order, [low, high], btype='band')
	return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
	b, a = butter_bandpass(lowcut, highcut, fs, order=order)
	y = lfilter(b, a, data)
	return y

def compute_DE(signal):
	variance = np.var(signal,ddof=1)
	return math.log(2*math.pi*math.e*variance)/2

def decompose(file_name):
	# trial*channel*sample
	start_index = 384 #3s pre-trial signals
	with open(file_name, 'rb') as file:
		subject = pickle.load(file, encoding='latin1')
	data = subject['data']
	shape = data.shape
	frequency = 128

	decomposed_de = np.empty([0,4,60])

	base_DE = np.empty([0,128])

	for trial in range(40):
		temp_base_DE = np.empty([0])
		temp_base_theta_DE = np.empty([0])
		temp_base_alpha_DE = np.empty([0])
		temp_base_beta_DE = np.empty([0])
		temp_base_gamma_DE = np.empty([0])

		temp_de = np.empty([0,60])

		for channel in range(32):
			trial_signal = data[trial,channel,384:]
			base_signal = data[trial,channel,:384]
			#****************compute base DE****************
			base_theta = butter_bandpass_filter(base_signal, 4, 8, frequency, order=3)
			base_alpha = butter_bandpass_filter(base_signal, 8,14, frequency, order=3)
			base_beta = butter_bandpass_filter(base_signal,14,31, frequency, order=3)
			base_gamma = butter_bandpass_filter(base_signal,31,45, frequency, order=3)

			base_theta_DE = (compute_DE(base_theta[:128])+compute_DE(base_theta[128:256])+compute_DE(base_theta[256:]))/3
			base_alpha_DE =(compute_DE(base_alpha[:128])+compute_DE(base_alpha[128:256])+compute_DE(base_alpha[256:]))/3
			base_beta_DE =(compute_DE(base_beta[:128])+compute_DE(base_beta[128:256])+compute_DE(base_beta[256:]))/3
			base_gamma_DE =(compute_DE(base_gamma[:128])+compute_DE(base_gamma[128:256])+compute_DE(base_gamma[256:]))/3

			temp_base_theta_DE = np.append(temp_base_theta_DE,base_theta_DE)
			temp_base_gamma_DE = np.append(temp_base_gamma_DE,base_gamma_DE)
			temp_base_beta_DE = np.append(temp_base_beta_DE,base_beta_DE)
			temp_base_alpha_DE = np.append(temp_base_alpha_DE,base_alpha_DE)

			theta = butter_bandpass_filter(trial_signal, 4, 8, frequency, order=3)
			alpha = butter_bandpass_filter(trial_signal, 8, 14, frequency, order=3)
			beta = butter_bandpass_filter(trial_signal, 14, 31, frequency, order=3)
			gamma = butter_bandpass_filter(trial_signal, 31, 45, frequency, order=3)

			DE_theta = np.zeros(shape=[0],dtype = float)
			DE_alpha = np.zeros(shape=[0],dtype = float)
			DE_beta =  np.zeros(shape=[0],dtype = float)
			DE_gamma = np.zeros(shape=[0],dtype = float)

			for index in range(60):
				DE_theta =np.append(DE_theta,compute_DE(theta[index*frequency:(index+1)*frequency]))
				DE_alpha =np.append(DE_alpha,compute_DE(alpha[index*frequency:(index+1)*frequency]))
				DE_beta =np.append(DE_beta,compute_DE(beta[index*frequency:(index+1)*frequency]))
				DE_gamma =np.append(DE_gamma,compute_DE(gamma[index*frequency:(index+1)*frequency]))
			temp_de = np.vstack([temp_de,DE_theta])
			temp_de = np.vstack([temp_de,DE_alpha])
			temp_de = np.vstack([temp_de,DE_beta])
			temp_de = np.vstack([temp_de,DE_gamma])
		temp_trial_de = temp_de.reshape(-1,4,60)
		decomposed_de = np.vstack([decomposed_de,temp_trial_de])

		temp_base_DE = np.append(temp_base_theta_DE,temp_base_alpha_DE)
		temp_base_DE = np.append(temp_base_DE,temp_base_beta_DE)
		temp_base_DE = np.append(temp_base_DE,temp_base_gamma_DE)
		base_DE = np.vstack([base_DE,temp_base_DE])
  
	decomposed_de = decomposed_de.reshape(-1,32,4,60).transpose([0,3,2,1]).reshape(-1,4,32).reshape(-1,128)
	print("base_DE shape:",base_DE.shape)
	print("trial_DE shape:",decomposed_de.shape)
	return base_DE,decomposed_de

def get_labels(file_name):
	with open(file_name, 'rb') as file:
		subject = pickle.load(file, encoding='latin1')
	labels = subject['labels']  
	#0 valence, 1 arousal, 2 dominance, 3 liking
	valence_labels = labels[:,0]>5	# valence labels
	arousal_labels = labels[:,1]>5	# arousal labels
	final_valence_labels = np.empty([0])
	final_arousal_labels = np.empty([0])
	for i in range(len(valence_labels)):
		for j in range(0,60):
			final_valence_labels = np.append(final_valence_labels,valence_labels[i])
			final_arousal_labels = np.append(final_arousal_labels,arousal_labels[i])
	print("labels:",final_arousal_labels.shape)
	return final_arousal_labels,final_valence_labels


In [ ]:
dataset_dir = "/content/drive/MyDrive/DEAP/"

result_dir = "/content/drive/MyDrive/DEAP_DE/"
if os.path.isdir(result_dir)==False:
  os.makedirs(result_dir)

for file in os.listdir(dataset_dir):
  if file == '.ipynb_checkpoints': continue 
  print("processing: ",file,"......")
  file_path = os.path.join(dataset_dir,file)
  base_DE,trial_DE = decompose(file_path)
  arousal_labels,valence_labels = get_labels(file_path)
  sio.savemat(result_dir+"DE_"+file[:-4]+'.mat',{"base_data":base_DE,"data":trial_DE,"valence_labels":valence_labels,"arousal_labels":arousal_labels})

processing:  s01.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s02.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s03.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s04.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s05.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s06.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s07.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s08.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s09.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s10.dat ......
base_DE shape: (40, 128)
trial_DE shape: (2400, 128)
labels: (2400,)
processing:  s11.dat ......
ba